In [21]:
from math import sqrt
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
from itertools import count
import networkx as nx
import time
from collections.abc import Iterable
from matplotlib import patches
import matplotlib.gridspec as gridspec

In [22]:
with open('./initializer/dataset_1.txt', 'r') as file:
    parameters = [int(x) for x in file.readline().split(' ')[:]]
    rows = [[int(x) for x in line.split(' ')[:]] for line in file]
    cols = [list(col) for col in zip(*rows)]

x_cords = cols[0]
y_cords = cols[1]
m_amount = cols[2]

n_islands, p_cost_factor, k_consicutive_islands, money_limit  = parameters[0], parameters[1], parameters[2], parameters[3]

cords_data = list(zip(x_cords, y_cords))

name2point = {i: j for i, j in zip(count(start=0, step=1), cords_data)}
point2name = {y: x for x,y in name2point.items()}

cords2money = {i: j for i, j in zip(cords_data, m_amount)}
island2money = {i: j for i, j in zip(count(start=0, step=1), m_amount)}

In [24]:
def Indexed_reachingCost(ip1, ip2):
    global cords_data
    p1 = cords_data[ip1]
    p2 = cords_data[ip2]
    return p_cost_factor * (sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2))

def factor_calc(currentIsland, nextIsland):
    pc_factor = island2money[nextIsland] - Indexed_reachingCost(nextIsland, currentIsland)
    return pc_factor

In [25]:
def BestFactor_circuit(cord):
    cord = cords_data
    circuit = list()
    start_vertex = 0
    dark_side = set(range(n_islands)) - {start_vertex}

    current_vertex = start_vertex
    while len(dark_side) > 0:
        max_factor = None
        best_v = None
        for v in dark_side:
            factor = factor_calc(current_vertex, v)
            if factor < 0:
                continue
            elif ((max_factor is None) or (max_factor < factor)):
                max_factor = factor
                best_v = v
        if best_v is not None:
            circuit.append((current_vertex, best_v))
            dark_side.remove(best_v)
            current_vertex = best_v

    circuit.append((current_vertex, start_vertex))

    circuitCost = sum(Indexed_reachingCost(*e) for e in circuit)
    
    OutputCircuit = [a for (a, b) in circuit]
    CircuitCords = [name2point[a] for a in OutputCircuit]
    circuitProfit = sum(cords2money[i] for i in CircuitCords) - island2money[0]

    netProfit = circuitProfit - circuitCost
    
    return OutputCircuit+[start_vertex], netProfit


In [23]:
def buildEdges(iterable):
    a, b = itertools.tee(iterable)
    next(b, None)
    return list(zip(a, b))

def tourCost(tour):
    edge_list = buildEdges(tour)
    totalCost = sum(Indexed_reachingCost(*e) for e in edge_list)
    return totalCost
    
def tourProfit(tour):
    CircuitCords = [name2point[a] for a in tour]
    totalProfit = sum(cords2money[i] for i in CircuitCords)
    return totalProfit

def edgesFactor(tour):
    factor = tourProfit(tour) - tourCost(tour)
    return factor

In [48]:
 def decision():
    mini = 0
    minj = 0
    for i in range(len(cords_data)-2):
        for j in range(i+2, len(cords_data)):
            minchange = factor_calc(i,i+1) + factor_calc(j,j+1)
            change = factor_calc(i,j) + factor_calc(i+1,j+1) - minchange
            if (minchange < change):
                minchange = change
                mini = i
                minj = j
    return mini, minj

In [26]:
def swapper (route, i, k, tourFactor):
    r1 = route[:i]
    r2 = route[i:k]
    r2 = r2[::-1]
    r3 = route[k:]
    
    new_route = r1+r2+r3
    return (new_route)

In [27]:
def super_duper():
    route, factor = BestFactor_circuit(cords_data)
    for i in range(len(cords_data)):
        for j in range(i,len(cords_data)):
            new_route, new_factor = swapper(route[:], i, j, factor)
            new_factor = factor + edgesFactor(route[decision()[0], decision[1]])
            if(new_factor>factor) and new_route[0] == new_route[-1] == 0:
                factor=new_factor
                route=new_route
            
    route = [a+1 for a in route]
    return route, tourFactor

In [49]:
decision()

KeyError: 980

In [28]:
id_tour, factor = super_duper()
hamiltonian_circuit = BestFactor_circuit(cords_data)[0]
cords_tour = []
for i in id_tour:
    cords_tour.append(name2point[i])
edge_list = list(zip(id_tour[:-1], id_tour[1:]))

KeyboardInterrupt: 

In [ ]:
plot_tour = [x + y for x, y in zip(cords_tour, cords_tour[1:])]
t = [(random.uniform(1, 3),) + i for i in plot_tour]

x = []
y = []
for point in cords_data:
    x.append(point[0])
    y.append(point[1])
    
data = np.array(t,dtype=[('MAG','int32'),('X0','int32'),('Y0','int32'),('X1','int32'),('Y1','int32')])

In [ ]:
plt.figure(figsize=(12,8)) 
plt.scatter(data['X0'], data['Y0'], color='r', zorder=10)
plt.scatter(data['X1'], data['Y1'], color='r', zorder=10)
all_islands = plt.scatter(x, y, color='DarkGreen', label='Group 1')
home_land = plt.scatter(cords_tour[0][0], cords_tour[0][1], color='Yellow', s=250, label='Group 2')

for d in data:
    if(d['MAG'] > 0):
        plt.arrow(d['X0'],d['Y0'],d['X1']-d['X0'], d['Y1']-d['Y0'], 
         shape='full', color='b', lw=d['MAG']/2., length_includes_head=True, 
         zorder=0, head_length=10., head_width=8.)
plt.show()

In [ ]:
x = [point[0] for point in cords_data]
y = [point[1] for point in cords_data]

totalProfit = factor
H_tour = []
for i in hamiltonian_circuit:
    H_tour.append(name2point[i])
H_EDGES = list(zip(hamiltonian_circuit[:-1], hamiltonian_circuit[1:]))

plot_hamiltonian = [x + y for x, y in zip(H_tour, H_tour[1:])]
q = [(random.uniform(1, 3),) + i for i in plot_hamiltonian]
data_H = np.array(q,dtype=[('MAG','float'),('X0','float'),('Y0','float'),('X1','float'),('Y1','float')])

In [ ]:
fig1 = plt.figure(constrained_layout=True, figsize = (35, 28))
spec2 = gridspec.GridSpec(ncols=3, nrows=3, figure=fig1)

f1_ax1 = fig1.add_subplot(spec2[0, 0])
x = [point[0] for point in cords_data]
y = [point[1] for point in cords_data]
rawIslands = f1_ax1.scatter(x, y, color='DarkGreen')
f1_ax1.set_title('Given Dataset', fontsize=15)
f1_ax1.annotate('total number of islands= {}'.format(n_islands), (0.02, 0.98), xycoords='axes fraction', va='center')

f1_ax2 = fig1.add_subplot(spec2[0, 1])
f1_ax2.scatter(data['X0'], data['Y0'], color='r', zorder=10)
f1_ax2.scatter(data['X1'], data['Y1'], color='r', zorder=10)
all_islands = f1_ax2.scatter(x, y, color='DarkGreen', label='Group 1')
homeLand = f1_ax2.scatter(cords_tour[0][0], cords_tour[0][1], color='LightGreen', s=250)
for d in data:
    if(d['MAG'] > 0):
        plt.arrow(d['X0'],d['Y0'],d['X1']-d['X0'], d['Y1']-d['Y0'], 
         shape='full', color='b', lw=d['MAG']/2., length_includes_head=True, 
         zorder=0, head_length=10., head_width=8.)
f1_ax2.set_title('Vanilla 2-OPT', fontsize=15)
f1_ax2.annotate('total NetProfit= {}'.format(totalProfit), (0.02, 0.98), xycoords='axes fraction', va='center')

f1_ax3 = fig1.add_subplot(spec2[0, 2])
f1_ax3.scatter(data_H['X0'], data_H['Y0'], color='r', zorder=10)
f1_ax3.scatter(data_H['X1'], data_H['Y1'], color='r', zorder=10)
all_islands = f1_ax3.scatter(x, y, color='DarkGreen', label='Group 1')
homeLand = f1_ax3.scatter(H_tour[0][0], H_tour[0][1], color='LightGreen', s=250)
for d in data_H:
    if(d['MAG'] > 0):
        plt.arrow(d['X0'],d['Y0'],d['X1']-d['X0'], d['Y1']-d['Y0'], 
         shape='full', color='b', lw=d['MAG']/2., length_includes_head=True, 
         zorder=0, head_length=10., head_width=8.)
f1_ax3.set_title('NN-Hamiltonian Circuit', fontsize=15)

f1_ax4 = fig1.add_subplot(spec2[1, 0])
x = np.asarray(x_cords)
y = np.asarray(y_cords)
m = np.asarray(m_amount)
f1_ax4 = plt.scatter(x, y, c = m, label='Group 1')

f1_ax5 = fig1.add_subplot(spec2[1, 1])
G = nx.DiGraph()
G.add_nodes_from(id_tour)
G.add_edges_from(edge_list)
edge_colors = ['black']
pos = nx.spiral_layout(G)
nx.draw(G, pos, node_size=22, ax=f1_ax5, edge_color=edge_colors, edge_cmap=plt.cm.Reds, with_labels = False)
f1_ax5.set_title('Cobweb Plot for Vanilla 2-OPT', fontsize=15)
f1_ax5.annotate('Visiting all of the {} islands'.format(len(id_tour)-1), (0.44, 0.98), xycoords='axes fraction', va='center')

f1_ax6 = fig1.add_subplot(spec2[1, 2])
G = nx.DiGraph()
G.add_nodes_from(hamiltonian_circuit)
G.add_edges_from(H_EDGES)
edge_colors = ['black']
pos = nx.spiral_layout(G)
nx.draw(G, pos, node_size=22, ax=f1_ax6, edge_color=edge_colors, edge_cmap=plt.cm.Reds, with_labels = False)
f1_ax6.set_title('Cobweb Plot for Hamiltonian', fontsize=15)
f1_ax6.annotate('Visiting all of the {} islands'.format(len(hamiltonian_circuit)-1), (0.44, 0.98), xycoords='axes fraction', va='center')

